# 科学技術計算 4 レポート


In [56]:
import numpy as np
from numpy.linalg import inv, norm, solve, det, matrix_rank, cond
from numpy import diag
rng = np.random.default_rng()

import scipy

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

import warnings
warnings.simplefilter('ignore', FutureWarning)

from typing import Tuple, List, Optional, Union
from tqdm.auto import tqdm

from numpy.linalg import solve


## 課題04-1

In [57]:
def lu_decomposition( A_org: np.ndarray ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """LU decomposition of A = PLU

    Args:
        A (np.ndarray): nxn matrix A

    Returns:
        (np.ndarray): nxn permutation matrix P
        (np.ndarray): nxn lower matrix L with 1s in diagonal
        (np.ndarray): nxn strictly upper matrix U
    """
    assert A_org.ndim == 2
    assert A_org.shape[0] == A_org.shape[1]

    #前進消去を行うコード
    n = A_org.shape[0]
    A = A_org.copy()
    print("A\n", A, sep="")
    L = np.eye(n)   #単位行列生成
    P = np.eye(n)   #置換行列

    for j in range(n - 1):
        #ここから資料にあった部分ピポッド選択月も前進消去の実装
          
        #print(f"{j}-th row", A[j])
        
        # 部分ピボット選択
        j_max = np.abs(A[j:, j]).argmax() + j   #行中の一番大きい値を抽出
        if j != j_max:
            # A の行交換
            A[[j, j_max]] = A[[j_max, j]]

            # L の行交換（前の列のみ）
            if j > 0:
                L[[j, j_max], :j] = L[[j_max, j], :j]

            # P の行交換 ← これが scipy と同じ P を作るカギ
            P[[j, j_max]] = P[[j_max, j]]
        
        #print("A\n", A, sep="")
        #print("P\n", P)
    
        # 前進消去
        remaining_rows = n - (j + 1)
        r_ij = A[j + 1:, j] / A[j, j]
        A[j + 1:, j + 1:] -= np.tile(A[j, j + 1:], (remaining_rows, 1)) * r_ij.reshape(remaining_rows, -1)
        A[j + 1:, j] = 0
        L[j + 1:, j] = r_ij
        #print("A\n", A, sep="")
        #$print("L\n", L, sep="")
        
    U = A
    
    #print("U\n", U, sep="")

    P = P.T
    """
    #確認１
    print("P =",P)
    print("L =", L)
    print("U =", U)

    print("LU\n", L @ U, sep="")
    print("A\n", A_org, sep="")
    #print("LU == A", np.allclose(L @ U, A_org))
    """
    return P, L, U


for i in tqdm(range(50)):
    n = rng.integers(low = 2, high = 10) #サイズ2-10
    A = rng.random(size=(n, n))

    P, L, U = lu_decomposition(A)
    Psp, Lsp, Usp = scipy.linalg.lu(A)
    """
    print("Psp = ", Psp)
    print("Lsp = ", Lsp)
    print("Usp = ", Usp)
    """
 
    assert np.allclose(L, Lsp), "values don't match"
    assert np.allclose(U, Usp), "values don't match"
    
    assert np.allclose(P, Psp), "values don't match"

  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.0316209  0.90786229 0.42782394 0.41800999 0.53440308 0.8320715
  0.9358932 ]
 [0.71511222 0.25860375 0.15474313 0.05347364 0.11968805 0.68963539
  0.22981707]
 [0.95857114 0.56525309 0.12525595 0.61566949 0.18598446 0.56633153
  0.36444046]
 [0.0131641  0.60019857 0.89519037 0.72018383 0.41569025 0.52759077
  0.04043314]
 [0.24429674 0.26018006 0.71214779 0.68452497 0.24743916 0.15462366
  0.04704701]
 [0.84646356 0.25718741 0.53594209 0.58347731 0.97791003 0.76134102
  0.47261461]
 [0.05026957 0.50595447 0.96930014 0.86638014 0.12465381 0.02464654
  0.16177531]]
A
[[0.41947087 0.9373814  0.78672999 0.79392763 0.40709165 0.36916597
  0.98397903 0.6583865  0.54999744]
 [0.07653816 0.61737545 0.01985922 0.05949649 0.68800712 0.29824346
  0.06021553 0.97826421 0.86659363]
 [0.28220281 0.31948179 0.54624728 0.02018462 0.7303068  0.78453635
  0.89197052 0.23297998 0.58970834]
 [0.88994917 0.78513065 0.44012737 0.53615301 0.31425678 0.13290516
  0.45609209 0.86965463 0.22845385]
 [0.67

## 課題04-2


In [58]:
def solve_lu( P: np.ndarray, L: np.ndarray, U: np.ndarray, b: np.ndarray,) -> np.ndarray:
    """Solve Ax = b with PLU, LU decomposition of A = PLU

    Args:
        P (np.ndarray): nxn permutation matrix P
        L (np.ndarray): nxn lower matrix L with 1s in diagonal
        U (np.ndarray): nxn strictly upper matrix U
        b (np.ndarray): n-d rhs vector b

    Returns:
        (np.ndarray): solution x of Ax = PLUx = b
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert P.shape == (n, n)
    assert L.shape == (n, n)
    assert U.shape == (n, n)
    b = b.copy()

    # code here
    # solve Ly = P^T b
    a = P.T @ b
    # 前進代入
    y = np.zeros(n)
    for i in range(n):
        y[i] = a[i] - np.dot(L[i, :i], y[:i])
        # L[i,i] = 1（単位下三角行列）なので割り算不要

    # 後退代入
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    """
    x_true = solve(A, b)
    print("true solution x", x_true)
    print("x == x_true", np.allclose(x_true, x))
    print(x - x_true)
    """

    return x

for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))
    b = rng.random(n)

    P, L, U = lu_decomposition(A)
    x_with_lu = solve_lu(P, L, U, b) #こっちが自作

    x_numpy = np.linalg.solve(A, b) #こっちがライブラリにあるやつ

    assert np.allclose(x_numpy, x_with_lu ), "values don't match"

  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.78529769 0.92980132 0.88919044]
 [0.26727335 0.5851111  0.6683768 ]
 [0.44309482 0.16347127 0.58755871]]
A
[[0.05696406 0.6856013  0.908327   0.5719096  0.21737023 0.03392291
  0.71010498 0.19330216 0.79126919]
 [0.19597561 0.75075804 0.44995136 0.68156566 0.96340144 0.57761487
  0.46478977 0.3358945  0.95849053]
 [0.06386185 0.05140732 0.11110407 0.45696749 0.19672062 0.48274221
  0.17567268 0.99634842 0.40532328]
 [0.55178146 0.49862161 0.99413556 0.80305983 0.75961623 0.39021725
  0.28262684 0.32033651 0.48367976]
 [0.61747278 0.59316299 0.35212251 0.45211152 0.0950062  0.53070237
  0.61908655 0.67186697 0.1312069 ]
 [0.52461363 0.58699763 0.55072241 0.74790426 0.4768552  0.17456094
  0.31765469 0.26258244 0.15874815]
 [0.91280173 0.42765035 0.871753   0.02468232 0.55392616 0.7393542
  0.54433225 0.29340644 0.59158656]
 [0.20947573 0.72478497 0.077683   0.15309118 0.37888467 0.01900474
  0.03231079 0.82907692 0.76045498]
 [0.6143272  0.95857369 0.4724978  0.91184862 0.14087612

## 課題04-4

In [83]:
#Pの行列式計算行う関数
def det_P(A: np.ndarray) -> float:
    
    """det(P) of permutation matrix P

    Args:
        P (np.ndarray): nxn permutation matrix of 0 and 1

    Returns:
        float: det(P) that is either 1 or -1
    """

    assert P.ndim == 2
    assert P.shape[0] == P.shape[1]

    for p in P.ravel()[np.nonzero(P.ravel())]:
        assert np.isclose(p, 1.0)
    
    n = P.shape[0]
    assert np.allclose(P.sum(axis=0), np.ones(n))
    assert np.allclose(P.sum(axis=1), np.ones(n))

    ## below three lines of code are helped with ChatGPT-4o because of its complexisity
    
    # Extract the permutation using np.nonzero
    permutation = np.nonzero(P)[1]

    # Calculate the number of inversions in the permutation
    inversions = sum(1 for i in range(len(permutation)) for j in range(i + 1, len(permutation)) if permutation[i] > permutation[j])

    # Return 1 if inversions are even, otherwise -1
    return 1 if inversions % 2 == 0 else -1


#Aの行列式をPLU分解を用いて行列式の計算行う関数
def det_lu( A_org: np.ndarray,) -> float:
    """compute det(A)

    Args:
        A (np.ndarray): nxn square matrix

    Returns:
        (float): det(A)
    """
    A = A_org.copy()
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    P, L, U = lu_decomposition(A)
    # code here
    #U求めちゃってかければハッピー(Lはスカラーだからたぶんどうにかなる)
    signP = det_P(P)   #Pの行列式の計算

    # U の対角成分の積（logで安全に計算
    diagU = np.diag(U) #対角成分だけとるここまであっている

    # 符号の計算
    log_symbol_U = np.prod(np.sign(diagU))
    

    # 対角絶対値の log の和
    log_abs_U = np.sum(np.log(np.abs(diagU))) #logをとる

    signU = log_symbol_U * np.exp(log_abs_U)
    detA = signP * signU

    return detA



#実行
for i in tqdm(range(10)):
    n = rng.integers(low=2, high=5)
    B = rng.random(size=(n, n))
    
    #行列式計算(自作関数)
    det_A = det_lu(A)
    #numpyのやつ
    A_numpy_det = np.linalg.det(A)
    

    print(det_A)
    print(A_numpy_det)

    assert np.isclose( det_A, A_numpy_det), "values don't match"


  0%|          | 0/10 [00:00<?, ?it/s]

A
[[0.24342865 0.03893253 0.60506921 0.51264315 0.94636178 0.00465122
  0.45129834 0.58332443]
 [0.71850735 0.01659715 0.08034376 0.38449817 0.06290599 0.30458775
  0.36191101 0.99914896]
 [0.68869397 0.53953224 0.81720742 0.91007385 0.0544553  0.26731779
  0.81461389 0.54607659]
 [0.04154714 0.24006032 0.43466476 0.25741422 0.35535588 0.04983317
  0.17241638 0.12038823]
 [0.26683322 0.46149853 0.79352637 0.11701786 0.94397548 0.45653024
  0.99708043 0.4627224 ]
 [0.30925371 0.76443727 0.28038816 0.77382998 0.76045688 0.51615084
  0.14818644 0.77467553]
 [0.66719433 0.15879487 0.80644477 0.00303688 0.99399288 0.27634242
  0.08244647 0.53072414]
 [0.74723452 0.86928249 0.81300442 0.11452332 0.23060026 0.50580456
  0.83807549 0.57160892]]
0.01841116428113163
0.01841116428113163
A
[[0.24342865 0.03893253 0.60506921 0.51264315 0.94636178 0.00465122
  0.45129834 0.58332443]
 [0.71850735 0.01659715 0.08034376 0.38449817 0.06290599 0.30458775
  0.36191101 0.99914896]
 [0.68869397 0.53953224 0

## 課題04-7

In [ ]:
#SOR
def sor(A: np.ndarray, b: np.ndarray, x0: np.ndarray | None = None, omega: float = 1.5, maxiter: int = 80, tol: float = 1e-8, callback: callable = None) -> np.ndarray:
    """SOR method for solving Ax=b

    Args:
        A (np.ndarray): nxn matrix A
        b (np.ndarray): n-d vector b
        xk (np.ndarray): n-d vector of initial value x0
        omega (float): omega of SOR
        maxiter (int, optional): max iterations. Defaults to 200.
        tol (float, optional): tolerance. Defaults to 1e-8.
        callback (callable, ``callback(diff: float, norm: float)``, optional): callback function. Defaults to None.

    Returns:
        (np.ndarray): n-d vector of the solution x
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert A.shape == (n, n)

    # code here
    if x0 is None:
        x0 = np.zeros(n)
    else:
        assert x0.shape == (n,)
    xk_1 = x0.copy()
    x_tilde_k = x0.copy()
    xk = x0.copy()

    D = diag(A) #対角行列
    LU = A.copy()
    np.fill_diagonal(LU, 0)

    for _ in range(maxiter):
        for i in range(n):
            x_tilde_k[i] = ( b[i] - (LU[i, :i] @ xk[:i] + LU[i, (i + 1):] @ xk_1[(i + 1):])) / D[i]
        for i in range(n):
            xk[i] = xk_1[i] + omega * (x_tilde_k[i] - xk_1[i])
    
        diff_k = norm(xk - xk_1)
        residual_k = norm(b - A @ xk)

        if callback:
            callback(diff_k, residual_k)

        if diff_k < tol:
            break
        xk_1 = xk.copy()
    return xk


#Jacobi call back含む
def jacobi_method( A: np.ndarray, b: np.ndarray, x0: np.ndarray | None = None, maxiter: int = 200, tol: float = 1e-8, callback: callable = None) -> np.ndarray:
    """Jacobi method for solving Ax=b

    Args:
        A (np.ndarray): nxn matrix A
        b (np.ndarray): n-d vector b
        x0 (np.ndarray): n-d vector of initial value x0
        maxiter (int, optional): max iterations. Defaults to 200.
        tol (float, optional): tolerance. Defaults to 1e-8.
        callback (callable, ``callback(diff: float, res: float)``, optional): callback function. Defaults to None.

    Returns:
        (np.ndarray): n-d vector of the solution x
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert A.shape == (n, n)

    if x0 is None:
        x0 = np.zeros(n)
    else:
        assert x0.shape == (n,)
    xk_1 = x0.copy()

    D = diag(A) #対角行列
    LU = A.copy()
    np.fill_diagonal(LU, 0) #対角成分が0になる

    for _ in range(maxiter):
        xk = (b - LU @ xk_1) / D

        diff_k = norm(xk - xk_1)
        residual_k = norm(b - A @ xk)

        if callback:
            callback(diff_k, residual_k)

        if diff_k < tol:
            break
        xk_1 = xk.copy()

    return xk

#Gauss Se1del
def gauss_seidel( A: np.ndarray, b: np.ndarray, x0: np.ndarray | None = None, maxiter: int = 50, tol: float = 1e-8, callback: callable = None) -> np.ndarray:
    """Gauss-Seidel method for solving Ax=b

    Args:
        A (np.ndarray): nxn matrix A
        b (np.ndarray): n-d vector b
        x0 (np.ndarray): n-d vector of initial value x0
        maxiter (int, optional): max iterations. Defaults to 200.
        tol (float, optional): tolerance. Defaults to 1e-8.
        callback (callable, ``callback(diff: float, res: float)``, optional): callback function. Defaults to None.

    Returns:
        (np.ndarray): n-d vector of the solution x
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert A.shape == (n, n)

    if x0 is None:
        x0 = np.zeros(n)
    else:
        assert x0.shape == (n,)
    xk_1 = x0.copy()
    xk = x0.copy()

    D = diag(A)
    LU = A.copy()
    np.fill_diagonal(LU, 0)

    for _ in range(maxiter):
        for i in range(n):
            xk[i] = ( b[i] - (LU[i, :i] @ xk[:i] + LU[i, (i + 1):] @ xk_1[(i + 1):])) / D[i]
            # xk[i] = (b[i] - LU[i] @ xk) / D[i]

        diff_k = norm(xk - xk_1)
        residual_k = norm(b - A @ xk)
        if callback:
            callback(diff_k, residual_k)

        if diff_k < tol:
            break
        xk_1 = xk.copy()

    return xk

#プロットするやつ
class CacheDiffRes(object):
    """
    callback class for caching differences and residuals
    """

    def __init__(self, name: str):
        """constructor

        Args:
            name (str): name of the cache
        """
        self.__diff_series = []
        self.__res_series = []
        self.__name = str(name)

    def __call__(self, diff: float, res: float) -> None:
        """function call of an instance of this class

        Args:
            diff (float): difference ||x_k - x_k-1||
            res (float): residual ||b - A x_k||
        """
        assert isinstance(diff, float) and isinstance(res, float)
        self.__diff_series.append(diff)
        self.__res_series.append(res)

    def get_diff(self) -> np.ndarray:
        """get a list of differences

        Returns:
            (np.ndarray): a copy of differences
        """
        return np.array(self.__diff_series)

    def get_res(self) -> np.ndarray:
        """get a list of residuals

        Returns:
            (np.ndarray): a copy of residuals
        """
        return np.array(self.__res_series)

    def get_name(self) -> str:
        """get name

        Returns:
            str: a copy of name
        """
        return str(self.__name)
        
        
#プロットするやつ2
class PlotCacheDiffRes(object):
    """Plot CacheDiffRes objects

    """

    def __init__(self, caches: List[CacheDiffRes]) -> None:
        """constructor

        Args:
            caches (List[CacheDiffRes]): list of CacheDiffRes objects
        """
        self.n_caches = len(caches)

        self.caches = caches

        self.x_label = "iterations k"
        self.y_label_diff = "$\\| x_k - x_{k-1} \\|$"
        self.y_label_res = "$\\| b - A x_k \\|$"

    def plot(self, y_logscale: bool = False) -> matplotlib.pyplot.figure:
        """Draw two plots of diffs and residuals over iterations

        Args:
            y_logscale (bool): flag to plot in log scale. Defaults to False.

        Returns:
            matplotlib.pyplot.figure: fig (used for fig.savefig())
        """
        fig = plt.figure()
        axs = fig.subplots(2, sharex=True)

        ax = axs[0]
        for cache in self.caches:
            ax.plot(cache.get_diff(), ".-", label=cache.get_name())
        ax.set_ylabel(self.y_label_diff)
        if y_logscale:
            ax.set_yscale("log")
        ax.legend()

        ax = axs[1]
        for cache in self.caches:
            ax.plot(cache.get_res(), ".-", label=cache.get_name())
        ax.set_ylabel(self.y_label_res)
        ax.set_xlabel(self.x_label)
        if y_logscale:
            ax.set_yscale("log")
        ax.legend()

        plt.show()

        return fig

#test
A = np.array([[1,2],
              [3,4]])
b = np.array([1,2])

y = np.linalg.solve(A,b)
cache_callback_gauss_seidel = CacheDiffRes("Gauss-Seidel")
x = gauss_seidel(A, b, callback=cache_callback_gauss_seidel)
print("x", x)
print("x == x_np", np.allclose(x, y))
"""cache_callback_sor = CacheDiffRes("SOR")
x = sor(A, b, callback=cache_callback_sor)
"""
print(f"numpy = {y}")
print(f"my ={x}")

"""#jacobiメソッドの実行
cache_callback_jacobi = CacheDiffRes("Jacobi method")
x = jacobi_method(A, b, callback=cache_callback_jacobi)
print("x", x)

plot_cache = PlotCacheDiffRes([
    cache_callback_jacobi
])


fig = plot_cache.plot()
fig = plot_cache.plot(y_logscale=True)
cache_callback_jacobi.get_diff()
cache_callback_jacobi.get_res()

#Gaussメソッドの実行
cache_callback_gauss_seidel = CacheDiffRes("Gauss-Seidel")
x = gauss_seidel(A, b, callback=cache_callback_gauss_seidel)
print("x", x)
x_np = solve(A, b)
print("x", x)
print("x_np", x_np)
print("x == x_np", np.allclose(x, x_np))
plot_cache = PlotCacheDiffRes(
    [cache_callback_gauss_seidel]
)

fig = plot_cache.plot()
fig = plot_cache.plot(y_logscale=True)

two_plots = PlotCacheDiffRes(
    [cache_callback_jacobi, cache_callback_gauss_seidel]
)
# fig = two_plots.plot()
fig = two_plots.plot(y_logscale=True)
"""
for i in range(10):
    n = rng.integers(low=2, high=10)
    print(n)
    A = rng.random(size=(n, n))
    b = rng.random(n)

    x_numpy = np.linalg.solve(A, b)
    cache_callback_sor = CacheDiffRes("SOR")
    x_mysor = sor(A, b,callback=cache_callback_sor)
    
    print(f"x_mysor = {x_mysor}")
    print(f"x_numpy = {x_numpy}")

    #assert np.isclose(x_numpy, x_mysor), "values doesn't match"



x [ 4.2508100e+08 -3.1881075e+08]
x == x_np False
numpy = [0.  0.5]
my =[ 4.2508100e+08 -3.1881075e+08]
7
x_mysor = [-2.46836872e+107 -1.39759894e+107 -1.06239138e+107 -4.18311257e+106
 -3.11396588e+106 -4.95870942e+107 -4.95518185e+106]
x_numpy = [-1.28605674  1.08262434 -0.00689389  0.50995051  1.8645966  -1.64864542
 -0.75619487]
9
x_mysor = [-3.89228426e+138 -4.60125309e+138 -5.57773883e+137 -4.53919607e+137
 -1.11037527e+137 -1.80786396e+137 -2.06509429e+137 -1.58624006e+138
 -2.07440579e+137]
x_numpy = [ 2.32249359  0.17479002 -3.27014412 -0.0197695   0.44076814 -1.40190661
  0.34209144 -0.47286694  1.93347947]
4
x_mysor = [-6.60238842e+35 -1.42117900e+35 -7.16523936e+35 -7.12522848e+35]
x_numpy = [ 6.47525693  1.71438225 -9.48145271  4.81683211]
8
x_mysor = [-9.71918210e+93 -8.45813418e+93 -5.44133561e+93 -1.25001839e+94
 -1.01260803e+94 -1.41645341e+94 -3.42803982e+94 -5.61985405e+94]
x_numpy = [  6.10106336  -2.48846131  -5.69254584  -8.27513311  15.07899446
 -13.87552244   7.